In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import glob
import json
import pickle as pkl
import ast
import os
from random import sample
import networkx as nx

from pandarallel import pandarallel
pandarallel.initialize()


INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
full_fdata = pd.read_csv("/data/shruti/ONR/small_data/facebook_linkpost_text.csv", header=0)

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (0,1,2,3,4,5,6,7,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
print("getting data")
# get random samples from each party of similar sizes
datafiles = glob.glob("/data/shruti/ONR/big_data/Twitter/*.csv")
plist = ['_BJP.csv', '_INC.csv', '_AAP.csv']
pcount = defaultdict(list)
for d in datafiles:
    for p in plist:
        if p in d:
            pcount[p].append(d)
                
final_files = pcount
#final_files = defaultdict(list)
# for p in pcount:
#     try:
#         final_files[p] = sample(pcount[p], 1000)
#     except:
#         final_files[p] = sample(pcount[p], 430)
    
    
#read randomly sampled data into a dataframe
rowlist = []
for p in final_files.keys():
    party = p.replace("_", "").replace(".csv", "")
    for d in final_files[p]:
        with open(d, "r") as jsonfile:
            for line in jsonfile:
                job= json.loads(line)
                if "text" in job:
                    text = job['text']
                else:
                    text = None
                if "screen_name" in job:
                    sn = job['screen_name']
                else:
                    sn = None
                    
                if "lang" in job:
                    language = job['lang']
                else:
                    language = None
                    
                row = [sn, text, party, language]
                rowlist.append(row)

                
fulltdata = pd.DataFrame(rowlist, columns=['screen_name','text','party', 'language'])
fulltdata = fulltdata.dropna(subset=['text'])

getting data


In [4]:
fulltdata = fulltdata.dropna(subset=['text'])

In [5]:
fulltdata = fulltdata.dropna(subset=['screen_name'])
len(fulltdata)

14023653

In [6]:
def remove_retweets(text):
    if "RT " in text:
    #if text.startswith("RT "):
        return 1

In [7]:
fulltdata['is_retweet'] = fulltdata['text'].parallel_apply(lambda x: remove_retweets(x))

In [8]:
fulltdata = fulltdata.loc[fulltdata['is_retweet']!=1.0]
len(fulltdata)

5672217

In [10]:
fulltdata.head()

,screen_name,text,party,language,is_retweet
100,Karan4BJP,@DivyaGuptabjp @vdsharmabjp @vaibhavpawarmp सभ...,BJP,hi,NaN
104,Karan4BJP,@JPNadda @ombirlakota आपके follow back की प्रत...,BJP,hi,NaN
105,Karan4BJP,How's it's ⁉️😅\nकैसी रही मित्रों..... https://...,BJP,hi,NaN
106,Karan4BJP,@Twitter Tonight you're eating followers in yo...,BJP,en,NaN
107,Karan4BJP,@SuhasBhagatBJP @TwitterIndia भाईसाहब सभी के उ...,BJP,hi,NaN


In [11]:
twitter_uniquetext = list(set(fulltdata['text'].tolist()))

In [12]:
full_fdata = full_fdata.dropna(subset=['handle','message'])

In [13]:
facebook_uniquetext = list(set(full_fdata['message'].tolist()))

In [15]:
len(twitter_uniquetext)

5464828

In [16]:
len(facebook_uniquetext)

6583067

In [17]:
common_texts = list(set(facebook_uniquetext).intersection(set(twitter_uniquetext)))

In [18]:
len(common_texts)

351

In [19]:
print(common_texts)

['भक्तों के सभी कष्टों को मुक्त कर देने वाली माँ धूमावती जयंती की हार्दिक शुभकामनाएं..', 'जय सियाराम।', 'डासना मंदिर में चोर घुसा, उसकी पिटाई हुई। महीनों विपक्ष का शोर मचा, मीडिया विलाप हुआ। दिल्ली की मस्जिद में 10 साल की बच्ची पानी पीने गई, उसका बलात्कार हो गया। अनोखा सा सन्नाटा है।', '😂', 'कोरोना से हुई मौत के आंकड़े छिपा रही है सरकारः कांग्रेस https://t.co/xU9zttRgza', 'विदेशी ताकत के संपर्क में राकेश टिकैत, संपत्ति की हो जांच- बीजेपी सांसद की मांग', 'सत्यमेव जयते।', 'तेजस्वी बोले- लालू जी ने 2014 में कहा था कि लोगों को तय करना है कि देश रहेगा या देश टूटेगा, आज देश की हालत सबके सामने है', 'HC ने केजरीवाल सरकार से सख्ती से पूछा है कि जो गरीबों का किराया देने का वादा किया था वो कब दिया जाएगा।CM ने दिल्ली की जनता को हर मोर्चे पर धोखा दिया है।दिल्ली की जनता से किये गए वादे पूरे होते नहीं और दूसरे राज्यों में घोषणा करने में लगे रहते हैं केजरीवाल। #KejriwalKeJhootheWade https://t.co/IBEZcuYEnk', 'Jai Hind', 'M', '👌👌', '#CourtSlamsKejriwal', 'भाई वीर सावरकर के साथ मिलकर हिंदू महासभा की नीं

In [67]:
short_twitter = fulltdata.loc[fulltdata['text'].isin(common_texts)]

short_facebook = full_fdata.loc[full_fdata['message'].isin(common_texts)]

print(len(short_twitter), len(short_facebook))

1248 4389


In [68]:
tframe = short_twitter[['screen_name','party','text']]
ffrmae = short_facebook[['handle','message']]

tframe = tframe.rename(columns={'screen_name':'handle', 'text':'message'})
ffrmae['party'] = 'facebook_group'

<ipython-input-68-313f89a17cc5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ffrmae['party'] = 'facebook_group'


In [69]:
tframe['handle'] = tframe['handle'].apply(lambda x: x + "_twitter")
ffrmae['handle'] = ffrmae['handle'].apply(lambda x: x + "_facebook")


<ipython-input-69-537e149a0ad7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ffrmae['handle'] = ffrmae['handle'].apply(lambda x: x + "_facebook")


In [70]:
frame = pd.concat([tframe, ffrmae])

In [71]:
frame.sample(10)

,handle,party,message
10905002,gargiMu48960835_twitter,INC,জয় শ্রী রাম
6476536,AtulgahlotBjp_twitter,BJP,जय श्री राम 🙏
3597663,shivsharmaIND_twitter,BJP,जय श्री राम
7036197,naveenjindalbjp_twitter,BJP,जय श्री राम
5964260,roshanrajak9934_twitter,BJP,#SaveBuxwahaForest
5749477,indiatvnews_facebook,facebook_group,"किसान हिंसा: अमित शाह की हाई लेवल मीटिंग शुरू,..."
2727173,latestInIndiaNews_facebook,facebook_group,.
8213169,latestInIndiaNews_facebook,facebook_group,.
4928155,sreenandaa_facebook,facebook_group,❤️
6505724,iRaviTiwari_twitter,BJP,जय श्री राम


In [72]:
def assign_unique_group(text):
    if len(text) > 15:
        uid = common_texts.index(text)
        return uid
    
frame['text_id'] = frame['message'].parallel_apply(lambda x: assign_unique_group(x))

In [73]:
len(frame)

5637

In [74]:
frame = frame.dropna(subset=['text_id'])

In [75]:
len(frame)

515

In [76]:
frame.head()

,handle,party,message,text_id
151469,dharmvir_9_twitter,BJP,Protests erupt in Pakistan's Gwadar amid growi...,93.0
510592,ambikeshkeshri_twitter,BJP,जय जय श्री राम 🚩🚩,154.0
819795,DhirajJhaBJP_twitter,BJP,Former UP CM #KalyanSingh passes away.,32.0
917821,ctex108_twitter,BJP,#TwitterBanInIndia,33.0
917835,ctex108_twitter,BJP,"डासना मंदिर में चोर घुसा, उसकी पिटाई हुई। महीन...",2.0


In [77]:
len("जय श्री राम")

11

In [78]:
frame.sample(5)

,handle,party,message,text_id
13608347,Ch_AnilKumarINC_twitter,INC,AAP दिल्ली की महिलाओं को भी दें एक हजार रुपये ...,222.0
16773552,IndiaTV.in_facebook,facebook_group,"राम मंदिर ट्रस्ट करेगा ऑक्सीजन की पूर्ति, दशरथ...",194.0
951168,Ramlal_twitter,BJP,ज्ञान हर व्यक्ति के जीवन का आधार है: डा० भीमरा...,57.0
13394324,iycnullipady_twitter,INC,#MyLeaderRahulGandhi,69.0
13610180,Ch_AnilKumarINC_twitter,INC,डीज़ल-पेट्रोल की कीमतों में बढ़ोतरी के खिलाफ क...,263.0


In [79]:
frame.loc[frame['text_id']==56.0]

,handle,party,message,text_id
4505328,rsbalawat_twitter,BJP,"बीजापुर, छत्तीसगढ में नक्सली हमले में शहीद हुए...",56.0
4711536,KailashBaytu_facebook,facebook_group,"बीजापुर, छत्तीसगढ में नक्सली हमले में शहीद हुए...",56.0


In [80]:
connections = frame.groupby('text_id')['handle'].apply(list).reset_index()
connections.head()

,text_id,handle
0,0.0,"[KailashOnline_twitter, KailashOnline_facebook]"
1,2.0,"[ctex108_twitter, sarafshubh23_twitter, Chanda..."
2,4.0,"[Ch_AnilKumarINC_twitter, INCDelhi_facebook]"
3,5.0,"[Harshad_BJP_twitter, TheJawansOfIndia_faceboo..."
4,7.0,"[PumpiINC_twitter, boltahindustan_facebook]"


In [81]:
handle2party = frame.set_index('handle').to_dict()['party']

In [87]:
nodes = defaultdict(int)
edges = defaultdict(int)

In [88]:
for idx, row in connections.iterrows():
    screenlist = list(set(row['handle']))
    for i in range(len(screenlist)):
        nodes[screenlist[i]]+=1
        
    for i in range(len(screenlist)-1):
        for j in range(i+1,len(screenlist)):
            stuple = sorted([screenlist[i], screenlist[j]])
            edges[(stuple[0], stuple[1])]+=1

In [89]:
edges

defaultdict(int,
            {('KailashOnline_facebook', 'KailashOnline_twitter'): 26,
             ('Chandanmishrbjp_twitter', 'sarafshubh23_twitter'): 1,
             ('Chandanmishrbjp_twitter', 'JagoReJago_facebook'): 1,
             ('Chandanmishrbjp_twitter', 'ctex108_twitter'): 1,
             ('JagoReJago_facebook', 'sarafshubh23_twitter'): 1,
             ('ctex108_twitter', 'sarafshubh23_twitter'): 1,
             ('JagoReJago_facebook', 'ctex108_twitter'): 1,
             ('Ch_AnilKumarINC_twitter', 'INCDelhi_facebook'): 19,
             ('Qweenranumondal_facebook',
              'RohitSardanaSudhirChaudharyFanClub_facebook'): 1,
             ('MubarakHoBetiHuyiHai_facebook',
              'RohitSardanaSudhirChaudharyFanClub_facebook'): 1,
             ('RohitSardanaSudhirChaudharyFanClub_facebook',
              'ShirdiWaleSaiBabaOfficial_facebook'): 1,
             ('JaiMataDiFounder_facebook',
              'RohitSardanaSudhirChaudharyFanClub_facebook'): 1,
             ('

In [90]:
G = nx.Graph()
for e in edges:
    
    if e[0] not in G.nodes:
        G.add_node(e[0], size=nodes[e[0]], party=handle2party[e[0]])
    if e[1] not in G.nodes:
        G.add_node(e[1], size=nodes[e[1]], party=handle2party[e[1]])

    G.add_edge(e[0], e[1], weight=edges[e])


In [91]:
nx.write_gexf(G, "../lite_data/astroturf_sametext_crossplatform.gexf", encoding='utf-8')